In [10]:
# import pakietów
import pandas as pd
import urllib
from IPython.display import HTML
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import re
from statistics import mean
import pickle

In [2]:
#import danych
col_names = ['ID', 'gender', 'city', 'income', 'birth_date', 'application_date', 'requested_amount', 
             'requested_period', 'financial_obligations', 'employer_name', 'account_bank', 'mobile_verification_flag',
             'var_5', 'var_1', 'offer_amount', 'offer_period', 'interest_rate', 'fee', 'offer_monthly_obligation',
             'filled_form_flag', 'device', 'var_2', 'source', 'var_4', 'logged_in_flag', 'disbursed_flag']
dataset = pd.read_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/train.csv', delimiter = ',', dtype='str', 
                      header = None, names = col_names, engine='python', index_col = False, skiprows = 1 )

HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000,5,0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000,2,0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000,4,0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000,5,0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000,2,25000,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000,2,nan,nan,nan,N,Web-browser,B,S134,3,1,0


In [3]:
months_dict = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
               'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

# TO DO - opis funckji i testy jednostkowe
def date_transform(date:str, twentieth_cent=False):
    day = date[:2]
    month = months_dict[date[3:6]]
    if twentieth_cent:
        return '19' + date[-2:] + "-" + month + "-" + day
    else:
        return '20' + date[-2:] + "-" + month + "-" + day

# formatowanie danych
dataset['income'] = dataset['income'].astype(float)
dataset['birth_date'] = dataset.apply(lambda x: date_transform(x['birth_date'], True), axis=1)
dataset['application_date'] = dataset.apply(lambda x: date_transform(x['application_date']), axis=1)
dataset['requested_amount'] = dataset['requested_amount'].astype(float)
dataset['financial_obligations'] = dataset['financial_obligations'].astype(float)

HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag
0,ID000002C20,Female,Delhi,20000.0,1978-05-23,2015-05-15,300000.0,5,0.0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000.0,1985-10-07,2015-05-04,200000.0,2,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500.0,1981-10-10,2015-05-19,600000.0,4,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000.0,1987-11-30,2015-05-09,1000000.0,5,0.0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000.0,1984-02-17,2015-05-20,500000.0,2,25000.0,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000,2,nan,nan,nan,N,Web-browser,B,S134,3,1,0


In [4]:
# gps coordinates of cities from Internet
min_latitude = 7
max_latitude = 38
min_longitude = 67
max_longitude = 98

def get_coords_from_pattern(string, pattern):
    coords = string.split(" ")
    degs = float(coords[0].split(pattern[6])[0])
    mins = float(coords[1].split(pattern[16])[0])
    secs = float(coords[2].split(pattern[33])[0])
    return degs + mins/60.0 + secs/3600.0


def convert_list_to_coord(found, pattern):
    n, e = None, None
    found_n, found_e = [], []
    # wiki pattern
    if 'nsew' not in pattern:
        found_n = [float(item.split(" ")[0].split('°')[0]) for item in found]
        found_e = [float(item.split(" ")[1].split('°')[0]) for item in found]
    # google patterns
    else:
        found_n = [get_coords_from_pattern(item, pattern) for item in found if item[-1]=='n']
        found_e = [get_coords_from_pattern(item, pattern) for item in found if item[-1]=='e']
        found_n = [coord for coord in found_n if coord >= min_latitude and coord <= max_latitude]
        found_e = [coord for coord in found_e if coord >= min_longitude and coord <= max_longitude]
    if found_n:
        n = mean(found_n)
    if found_e:
        e = mean(found_e)               
    return [n, e]


# 1 - data directly from Wikipedia
def get_wikipedia_data(city):
    pattern = "[0-9]+.[0-9]+°n [0-9]+.[0-9]+°e"
    found = []
    try:
        with urlopen("https://en.wikipedia.org/wiki/" + city) as page:
            soup = BeautifulSoup(page.read(), "html.parser")
            response_list = soup.select(".geo-dec")            
            for item in response_list:
                found += re.findall(pattern, item.get_text().lower())
    except urllib.request.HTTPError:
        pass
    return convert_list_to_coord(found, pattern)


# 2 - data from google search results   
def get_google_search_data(city):
    pattern = "[0-9]+°[ ][0-9]+'[ ][0-9]+.[0-9]+''[ ][nsew]"
    found = []        
    driver.get('https://www.google.pl/search?q=' + city + '+latitude+and+longitude+in+decimal')
    found = re.findall(pattern, driver.page_source.lower())
    return convert_list_to_coord(found, pattern)


def get_latitude_to_data(city):
    pattern = "[0-9]+°[ ][0-9]+'[ ][0-9]+.[0-9]+\"[ ][nsew]"
    found = []        
    driver.get('https://www.google.pl/search?q=' + city + '+india+gps+coords')
    try:
        links = driver.find_elements_by_css_selector('.rc')
        links = [item for item in links if 'latitude.to' in item.text.lower()]
        if links:
            links[0].find_element_by_css_selector('h3').click()
            undone = True
            while undone:
                if 'google' not in driver.current_url:
                    try:
                        found = re.findall(pattern, driver.page_source.lower())
                    except NoSuchElementException:
                        pass                        
                    undone = False
    except NoSuchElementException:
        pass
    return convert_list_to_coord(found, pattern)

In [6]:
# cities = sorted(list(set(dataset['city'])))
# driver = webdriver.Chrome('chromedriver')

# coords = {}
# for item in cities:
#     coords[item] = get_wikipedia_data(item)
#     if coords[item][0] == None or coords[item][1] == None:
#         coords[item] = get_google_search_data(item)
#     if coords[item][0] == None or coords[item][1] == None:
#         coords[item] = get_latitude_to_data(item)   
# driver.close()

# cities_coords = pd.DataFrame.from_dict(coords, orient='index', columns = ['city', 'coords'])
# cities_coords.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/gps_coords.csv')
cities_coords = pd.read_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/gps_coords.csv', names = ['city', 'latitude', 'longitude'], skiprows = 1)
cities_coords = cities_coords[~pd.isnull(cities_coords['latitude'])]
dataset2 = pd.merge(dataset, cities_coords, on = 'city', how='left')

In [7]:
# income - replacing outliers with median
percentile_level = 99
import statistics as stat
import numpy as np

incomes = list(dataset2['income'])
incomes_count = len(incomes)
incomes_mean = stat.mean(incomes)
incomes_median = stat.median(incomes)
incomes_stddev = stat.stdev(incomes)
incomes_min = min(incomes)
incomes_max = max(incomes)
incomes_perc_1 = np.percentile(dataset2['income'], 1, axis=0)
incomes_perc_up = np.percentile(dataset2['income'], percentile_level, axis=0)

print("n \t" + str(incomes_count))
print("min \t" + str(incomes_min))
print("perc_1 \t" + str(incomes_perc_1))
print("median \t" + str(incomes_median))
print("mean \t" + str(incomes_mean))
print("stddev \t" + str(incomes_stddev))
print("perc_up\t" + str(incomes_perc_up))
print("max \t" + str(incomes_max) + "\n")
income_outliers = [income for income in incomes if income < incomes_perc_1 or income > incomes_perc_up]
print('number of outliers: ' + str(len(income_outliers)) + "\n")
incomes = [income if income >= incomes_perc_1 and income <= incomes_perc_up else incomes_median for income in incomes]
dataset2['income'] = dataset.apply(lambda x: x['income'] if incomes_perc_up >= x['income'] >= incomes_perc_1 else incomes_median, axis = 1)

incomes = list(dataset2['income'])
incomes_count = len(incomes)
incomes_mean = stat.mean(incomes)
incomes_median = stat.median(incomes)
incomes_stddev = stat.stdev(incomes)
incomes_min = min(incomes)
incomes_max = max(incomes)

print("n \t" + str(incomes_count))
print("min \t" + str(incomes_min))
print("perc_1 \t" + str(np.percentile(dataset2['income'], 1, axis=0)))
print("median \t" + str(incomes_median))
print("mean \t" + str(incomes_mean))
print("stddev \t" + str(incomes_stddev))
print("perc_up\t" + str(np.percentile(dataset2['income'], percentile_level, axis=0)))
print("max \t" + str(incomes_max) + "\n")

n 	87020
min 	0.0
perc_1 	2583.0
median 	25000.0
mean 	58849.97435072397
stddev 	2177511.3613959304
perc_up	250000.0
max 	444554443.0

number of outliers: 1687

n 	87020
min 	2583.0
perc_1 	7500.0
median 	25000.0
mean 	32982.930016088256
stddev 	28592.258389480143
perc_up	170000.0
max 	250000.0



In [8]:
# age at the moment of application

def get_date_diff_in_years(date_start, date_end):
    out  = int(date_end[:4]) - int(date_start[:4])
    out += (int(date_end[5:7]) - int(date_start[5:7]))/12.0
    out += (int(date_end[8:]) - int(date_start[8:]))/30.5
    return round(out, 3)

dataset2['age'] = dataset2.apply(lambda x: get_date_diff_in_years(x['birth_date'], x['application_date']), axis=1)
HTML(dataset2.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag,latitude,longitude,age
0,ID000002C20,Female,Delhi,20000.0,1978-05-23,2015-05-15,300000.0,5,0.0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,0,28.61000,77.23000,36.738
1,ID000004E40,Male,Mumbai,35000.0,1985-10-07,2015-05-04,200000.0,2,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,0,18.97500,72.82583,29.485
2,ID000007H20,Male,Panchkula,22500.0,1981-10-10,2015-05-19,600000.0,4,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0,0,30.74000,76.80000,33.878
3,ID000008I30,Male,Saharsa,35000.0,1987-11-30,2015-05-09,1000000.0,5,0.0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0,0,25.88000,86.60000,26.811
4,ID000009J40,Male,Bengaluru,100000.0,1984-02-17,2015-05-20,500000.0,2,25000.0,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000,2,nan,nan,nan,N,Web-browser,B,S134,3,1,0,12.97575,77.57325,31.348


In [12]:
dataset2.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/datamata.csv', sep = ';')
with open('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/dataset2.pickle', 'wb') as file:
    pickle.dump(dataset2, file)

In [15]:
# Start from here:

with open('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/dataset2.pickle', 'rb') as file:
    dataset2 = pickle.load(file)
dataset2.shape

(87020, 29)

In [16]:
#requested_amount check
requested_amount_is_null = dataset2[pd.isnull(dataset2['requested_amount'])]
requested_amount_is_not_null = dataset2[~pd.isnull(dataset2['requested_amount'])]

requested_amount = list(requested_amount_is_not_null['requested_amount'])
requested_amount_count = len(requested_amount)
requested_amount_min = min(requested_amount)
requested_amount_mean = stat.mean(requested_amount)
requested_amount_median = stat.median(requested_amount)
requested_amount_stddev = stat.stdev(requested_amount)
requested_amount_max = max(requested_amount)

print("n \t" + str(requested_amount_count))
print("min \t" + str(requested_amount_min))
print("median \t" + str(requested_amount_median))
print("mean \t" + str(requested_amount_mean))
print("stddev \t" + str(requested_amount_stddev))
print("max \t" + str(requested_amount_max) + "\n")

requested_amount_is_null['requested_amount'] = requested_amount_median
dataset2 = pd.concat([requested_amount_is_null, requested_amount_is_not_null], axis=0)

n 	86949
min 	0.0
median 	100000.0
mean 	230250.69992754373
stddev 	354206.7594681699
max 	10000000.0



C:\Users\Ja\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
#requested_period check

requested_period = dataset2['requested_period']
print(set(requested_period))

requested_period_is_null = dataset2[dataset2['requested_period']=='nan']
requested_period_is_not_null = dataset2[dataset2['requested_period']!='nan']
requested_period_is_not_null['requested_period'] = requested_period_is_not_null['requested_period'].astype(int)
requested_period = list(requested_period_is_not_null['requested_period'])
requested_period_median = stat.median(requested_period)

requested_period_is_null['requested_period'] = requested_period_median
dataset2 = pd.concat([requested_period_is_null, requested_period_is_not_null], axis=0)

{'2', '5', '8', '9', '6', '7', '4', '1', 'nan', '10', '3', '0'}


C:\Users\Ja\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Ja\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [18]:
#financial_obligations check
financial_obligations_is_null = dataset2[pd.isnull(dataset2['financial_obligations'])]
financial_obligations_is_not_null = dataset2[~pd.isnull(dataset2['financial_obligations'])]

financial_obligations = list(financial_obligations_is_not_null['financial_obligations'])
financial_obligations_count = len(financial_obligations)
financial_obligations_min = min(financial_obligations)
financial_obligations_perc_1 = np.percentile(financial_obligations, 1, axis=0)
financial_obligations_mean = stat.mean(financial_obligations)
financial_obligations_median = stat.median(financial_obligations)
financial_obligations_stddev = stat.stdev(financial_obligations)
financial_obligations_perc_99 = np.percentile(financial_obligations, 99, axis=0)
financial_obligations_max = max(financial_obligations)


print("n \t" + str(financial_obligations_count))
print("min \t" + str(financial_obligations_min))
print("perc 1% \t" + str(financial_obligations_perc_1))
print("median \t" + str(financial_obligations_median))
print("mean \t" + str(financial_obligations_mean))
print("stddev \t" + str(financial_obligations_stddev))
print("perc 99% \t" + str(financial_obligations_perc_99))
print("max \t" + str(financial_obligations_max) + "\n")

financial_obligations_positive = [x for x in financial_obligations if x > 0]
financial_obligations_positive_count = len(financial_obligations_positive)
financial_obligations_positive_min = min(financial_obligations_positive)
financial_obligations_positive_perc_1 = np.percentile(financial_obligations_positive, 1, axis=0)
financial_obligations_positive_mean = stat.mean(financial_obligations_positive)
financial_obligations_positive_median = stat.median(financial_obligations_positive)
financial_obligations_positive_stddev = stat.stdev(financial_obligations_positive)
financial_obligations_positive_perc_99 = np.percentile(financial_obligations_positive, 99, axis=0)
financial_obligations_positive_max = max(financial_obligations_positive)

print("\nStatistics for positive values subset:\n")
print("n \t" + str(financial_obligations_positive_count))
print("min \t" + str(financial_obligations_positive_min))
print("perc 1% \t" + str(financial_obligations_positive_perc_1))
print("median \t" + str(financial_obligations_positive_median))
print("mean \t" + str(financial_obligations_positive_mean))
print("stddev \t" + str(financial_obligations_positive_stddev))
print("perc 99% \t" + str(financial_obligations_positive_perc_99))
print("max \t" + str(financial_obligations_positive_max) + "\n")

financial_obligations_positive = [x for x in financial_obligations if x > 0 and x < 100000]
financial_obligations_positive_count = len(financial_obligations_positive)
financial_obligations_positive_min = min(financial_obligations_positive)
financial_obligations_positive_perc_1 = np.percentile(financial_obligations_positive, 1, axis=0)
financial_obligations_positive_mean = stat.mean(financial_obligations_positive)
financial_obligations_positive_median = stat.median(financial_obligations_positive)
financial_obligations_positive_stddev = stat.stdev(financial_obligations_positive)
financial_obligations_positive_perc_99 = np.percentile(financial_obligations_positive, 99, axis=0)
financial_obligations_positive_max = max(financial_obligations_positive)

print("\nStatistics for positive values subset after removing outliers:\n")
print("n \t" + str(financial_obligations_positive_count))
print("min \t" + str(financial_obligations_positive_min))
print("perc 1% \t" + str(financial_obligations_positive_perc_1))
print("median \t" + str(financial_obligations_positive_median))
print("mean \t" + str(financial_obligations_positive_mean))
print("stddev \t" + str(financial_obligations_positive_stddev))
print("perc 99% \t" + str(financial_obligations_positive_perc_99))
print("max \t" + str(financial_obligations_positive_max) + "\n")

financial_obligations_is_null['financial_obligations'] = 0
financial_obligations_is_not_null['financial_obligations'] = financial_obligations_is_not_null.apply(lambda x: x['financial_obligations'] if x['financial_obligations'] < 100000 else 100000, axis = 1)
dataset2 = pd.concat([financial_obligations_is_null, financial_obligations_is_not_null], axis=0)

n 	86949
min 	0.0
perc 1% 	0.0
median 	0.0
mean 	3696.227823896767
stddev 	39810.21191986883
perc 99% 	40552.00000000041
max 	10000000.0


Statistics for positive values subset:

n 	28711
min 	1.0
perc 1% 	48.0
median 	6000.0
mean 	11193.734563756052
stddev 	68671.58242688209
perc 99% 	65000.0
max 	10000000.0


Statistics for positive values subset after removing outliers:

n 	28615
min 	1.0
perc 1% 	48.0
median 	6000.0
mean 	10199.236381618033
stddev 	11231.444213324834
perc 99% 	56000.0
max 	99500.0



C:\Users\Ja\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ja\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
dataset2.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/dataset3.csv', sep = ';')
with open('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/dataset3.pickle', 'wb') as file:
    pickle.dump(dataset2, file)

In [22]:
# Start from here:
with open('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/dataset3.pickle', 'rb') as file:
    dataset3 = pickle.load(file)
HTML(dataset3.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag,latitude,longitude,age
42858,ID061421J10,Male,Barnala,40000.0,1979-02-15,2015-06-18,100000.0,2,0.0,nan,nan,Y,8,HBXC,100000,2,16.35,1000,4913.05,Y,Web-browser,B,S157,5,0,0,NaN,NaN,36.432
43057,ID061704G40,Male,Ghaziabad,40000.0,1983-02-22,2015-06-19,100000.0,2,0.0,nan,nan,Y,13,HCXF,100000,2,17.5,2000,4968.28,Y,Web-browser,B,S143,5,0,0,28.663215,77.448275,32.235
48356,ID069305P00,Male,Pune,25500.0,1992-07-03,2015-06-24,100000.0,2,0.0,nan,nan,Y,8,HBXB,100000,3,27,2000,4082.52,Y,Web-browser,B,S133,5,0,0,18.520280,73.856670,23.605
49137,ID070459Z40,Female,Thane,38000.0,1988-06-08,2015-06-25,100000.0,2,0.0,nan,nan,N,10,HBXX,nan,nan,nan,nan,nan,N,Web-browser,D,S122,7,0,0,19.172431,72.957019,27.557
50104,ID071821J10,Male,Pune,38000.0,1987-10-31,2015-06-26,100000.0,2,0.0,nan,nan,Y,14,HBXB,400000,4,23.5,8000,12930.41,Y,Web-browser,B,S143,5,0,0,18.520280,73.856670,27.503


In [32]:
gender = dataset3[['gender', 'disbursed_flag']]
gender['disbursed_flag'] = gender['disbursed_flag'].astype(int)
gender_aggr = gender.groupby(['gender']).mean()
gender_aggr

C:\Users\Ja\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,disbursed_flag
gender,
Female,0.008555
Male,0.019158


In [ ]:
# gender parametryzuję tak - kobieta = 0, mężczyzna = 1 --> wg rosnącego udziału disbursed


In [37]:
dataset3['M01_gender'] = dataset3.apply(lambda x: 0 if x['gender']=='Female' else 1, axis=1)
HTML(dataset3.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag,latitude,longitude,age,M01_gender
42858,ID061421J10,Male,Barnala,40000.0,1979-02-15,2015-06-18,100000.0,2,0.0,nan,nan,Y,8,HBXC,100000,2,16.35,1000,4913.05,Y,Web-browser,B,S157,5,0,0,NaN,NaN,36.432,1
43057,ID061704G40,Male,Ghaziabad,40000.0,1983-02-22,2015-06-19,100000.0,2,0.0,nan,nan,Y,13,HCXF,100000,2,17.5,2000,4968.28,Y,Web-browser,B,S143,5,0,0,28.663215,77.448275,32.235,1
48356,ID069305P00,Male,Pune,25500.0,1992-07-03,2015-06-24,100000.0,2,0.0,nan,nan,Y,8,HBXB,100000,3,27,2000,4082.52,Y,Web-browser,B,S133,5,0,0,18.520280,73.856670,23.605,1
49137,ID070459Z40,Female,Thane,38000.0,1988-06-08,2015-06-25,100000.0,2,0.0,nan,nan,N,10,HBXX,nan,nan,nan,nan,nan,N,Web-browser,D,S122,7,0,0,19.172431,72.957019,27.557,0
50104,ID071821J10,Male,Pune,38000.0,1987-10-31,2015-06-26,100000.0,2,0.0,nan,nan,Y,14,HBXB,400000,4,23.5,8000,12930.41,Y,Web-browser,B,S143,5,0,0,18.520280,73.856670,27.503,1


In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.diagnostic import lilliefors
import math
power1 = 0.5
power2 = 0.3
power3 = 0.1
power4 = 0.01
num_bins = 50

incomes_norm = [income / incomes_median for income in incomes]
incomes_log  = [math.log(income) for income in incomes]
incomes_pow1 = [math.pow(income, power1) for income in incomes]
incomes_pow2 = [math.pow(income, power2) for income in incomes]
incomes_pow3 = [math.pow(income, power3) for income in incomes]
incomes_pow4 = [math.pow(income, power4) for income in incomes]

# n, bins, patches = plt.hist(incomes_sqrt, bins=40, facecolor='g')
# plt.show()

plt.figure()
mu_1, sigma_1 = stats.norm.fit(incomes_norm)
incomes_norm = [ (income - mu_1)/math.pow(sigma_1, 0.5) for income in incomes_norm]
mu_1, sigma_1 = stats.norm.fit(incomes_norm)
t1 = np.arange(min(incomes_norm), max(incomes_norm), 0.001)
plt.hist(incomes_norm, bins = num_bins, density=True, histtype='stepfilled', alpha=0.75, label='normalized')
plt.plot(t1, stats.norm.pdf(t1, mu_1, sigma_1), 'k-', lw=2, label='a=-1, b=1')
plt.show()

pVals = pd.Series()
_, pVals['Omnibus']    = stats.normaltest(incomes_norm)
_, pVals['Shapiro-Wilk']    = stats.shapiro(incomes_norm)
_, pVals['Lilliefors']    = lilliefors(incomes_norm)
print('p-values for all {0} data points: ----------------'.format(len(incomes_norm)))
print(pVals)


plt.figure()
mu_2, sigma_2 = stats.norm.fit(incomes_log)
incomes_log = [ (income - mu_2)/math.pow(sigma_2, 0.5) for income in incomes_log]
plt.hist(incomes_log, bins = num_bins, density=True, histtype='stepfilled', alpha=0.75, label='logarithm')
t2 = np.arange(min(incomes_log), max(incomes_log), 0.0001)
mu_2, sigma_2 = stats.norm.fit(incomes_log)
plt.plot(t2, stats.norm.pdf(t2, mu_2, sigma_2), 'k-', lw=2, label='a=-1, b=1')
plt.show()

pVals = pd.Series()
_, pVals['Omnibus']    = stats.normaltest(incomes_log)
_, pVals['Shapiro-Wilk']    = stats.shapiro(incomes_log)
_, pVals['Lilliefors']    = lilliefors(incomes_log)
print('p-values for all {0} data points: ----------------'.format(len(incomes_log)))
print(pVals)

plt.figure()
plt.hist(incomes_pow1, bins = num_bins, density=True, histtype='stepfilled', alpha=0.75, label='power ' + str(power1))
t3 = np.arange(min(incomes_pow1), max(incomes_pow1), 0.01)
mu_3, sigma_3 = stats.norm.fit(incomes_pow1)
plt.plot(t3, stats.norm.pdf(t3, mu_3, sigma_3), 'k-', lw=2, label='a=-1, b=1')
plt.show()

pVals = pd.Series()
_, pVals['Omnibus']    = stats.normaltest(incomes_pow1)
_, pVals['Shapiro-Wilk']    = stats.shapiro(incomes_pow1)
_, pVals['Lilliefors']    = lilliefors(incomes_pow1)
print('p-values for all {0} data points: ----------------'.format(len(incomes_pow1)))
print(pVals)


plt.figure()
plt.hist(incomes_pow2, bins = num_bins, density=True, histtype='stepfilled', alpha=0.75, label='power 0.3')
t4 = np.arange(min(incomes_pow2), max(incomes_pow2), 0.01)
mu_4, sigma_4 = stats.norm.fit(incomes_pow2)
plt.plot(t4, stats.norm.pdf(t4, mu_4, sigma_4), 'k-', lw=2, label='a=-1, b=1')
plt.show()

pVals = pd.Series()
_, pVals['Omnibus']    = stats.normaltest(incomes_pow2)
_, pVals['Shapiro-Wilk']    = stats.shapiro(incomes_pow2)
_, pVals['Lilliefors']    = lilliefors(incomes_pow2)
print('p-values for all {0} data points: ----------------'.format(len(incomes_pow2)))
print(pVals)


plt.figure()
plt.hist(incomes_pow3, bins = num_bins, density=True, histtype='stepfilled', alpha=0.75, label='power 0.1')
t5 = np.arange(min(incomes_pow3), max(incomes_pow3), 0.01)
mu_5, sigma_5 = stats.norm.fit(incomes_pow3)
plt.plot(t5, stats.norm.pdf(t5, mu_5, sigma_5), 'k-', lw=2, label='a=-1, b=1')
plt.show()

pVals = pd.Series()
_, pVals['Omnibus']    = stats.normaltest(incomes_pow3)
_, pVals['Shapiro-Wilk']    = stats.shapiro(incomes_pow3)
_, pVals['Lilliefors']    = lilliefors(incomes_pow3)
print('p-values for all {0} data points: ----------------'.format(len(incomes_pow3)))
print(pVals)


plt.figure()
plt.hist(incomes_pow4, bins = num_bins, density=True, histtype='stepfilled', alpha=0.75, label='power 0.01')
t6 = np.arange(min(incomes_pow4), max(incomes_pow4), 0.001)
mu_6, sigma_6 = stats.norm.fit(incomes_pow4)
plt.plot(t6, stats.norm.pdf(t6, mu_6, sigma_6), 'k-', lw=2, label='a=-1, b=1')
plt.show()

pVals = pd.Series()
_, pVals['Omnibus']    = stats.normaltest(incomes_pow4)
_, pVals['Shapiro-Wilk']    = stats.shapiro(incomes_pow4)
_, pVals['Lilliefors']    = lilliefors(incomes_pow4)
print('p-values for all {0} data points: ----------------'.format(len(incomes_pow4)))
print(pVals)


# import matplotlib.pyplot as plt
# plt.figure(1)                # the first figure
# plt.subplot(211)             # the first subplot in the first figure
# plt.plot([1, 2, 3])
# plt.subplot(212)             # the second subplot in the first figure
# plt.plot([4, 5, 6])


# plt.figure(2)                # a second figure
# plt.plot([4, 5, 6])          # creates a subplot(111) by default

# plt.figure(1)                # figure 1 current; subplot(212) still current
# plt.subplot(211)             # make subplot(211) in figure1 current
# plt.title('Easy as 1, 2, 3') # subplot 211 title




# fig, ax = plt.subplots(1, 3, squeeze=True)
# fig1 = ax[0].hist(incomes_norm, bins = num_bins, normed=True, histtype='stepfilled', alpha=0.75, label='normalized')
# fig1[0]=12
# ax[1].hist(incomes_log, bins = num_bins, normed=True, histtype='stepfilled', alpha=0.75, label='logarithm')
# ax[2].hist(incomes_pow1, bins = num_bins, normed=True, histtype='stepfilled', alpha=0.75, label='power ' + str(power1))
# plt.show()
# fig, ax = plt.subplots(1, 3, sharey=True)
# ax[0].hist(incomes_pow2, bins = num_bins, normed=True, histtype='stepfilled', alpha=0.75, label='power 0.3')
# ax[1].hist(incomes_pow3, bins = num_bins, normed=True, histtype='stepfilled', alpha=0.75, label='power 0.1')
incomes# ax[2].hist(incomes_pow4, bins = num_bins, normed=True, histtype='stepfilled', alpha=0.75, label='power 0.01')

In [ ]:
N = 100
params_list = range(0, N)

for pow in params_list:
#     print("param = " + str(pow) + "\n")
    pow = pow / N
    incomes_test = incomes
    incomes_test = stats.boxcox(incomes_test, lmbda = pow)
    mu, sigma = stats.norm.fit(incomes_test)
#     print("mu = " + str(mu) + " sigma = " + str(sigma))
#     print("\nNormalizacja\n")
    incomes_test = [(income - mu)/sigma for income in incomes_test]
    mu, sigma = stats.norm.fit(incomes_test)
#     print("mu = " + str(mu) + " sigma = " + str(sigma))

#     plt.figure()
#     t = np.arange(min(incomes_test), max(incomes_test), 0.001)
#     plt.hist(incomes_test, bins = num_bins, density=True, histtype='stepfilled', alpha=0.75, label='normalized')
#     plt.plot(t, stats.norm.pdf(t, mu, sigma), 'k-', lw=1,)
#     plt.show()

    pVals = pd.Series()
    _, pVals['Omnibus'] = stats.normaltest(incomes_test)
    _, pVals['Shapiro-Wilk'] = stats.shapiro(incomes_test)
    _, pVals['Lilliefors'] = lilliefors(incomes_test)
#     print('p-values for all {0} data points: ----------------'.format(len(incomes_test)))
#     print(pVals)
    print(str(pow) + "\t" + str(pVals['Omnibus']) + "\t" + str(pVals['Shapiro-Wilk']) + "\t" + str(pVals['Lilliefors']))

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.cluster.vq import whiten
from scipy.cluster.vq import kmeans, vq
import seaborn as sns


In [ ]:
cities_cluster = dataset2[['city', 'latitude', 'longitude']]
cities_cluster["scaled_latitude"] = whiten(cities_cluster["latitude"])
cities_cluster["scaled_longitude"] = whiten(cities_cluster["longitude"])

dist = []
num_clusters = range(1, 10)
for i in num_clusters:
    cluster_centers, distortion = kmeans(cities_cluster[["scaled_latitude","scaled_longitude"]],i)
    dist.append(distortion)

elbow_plot = pd.DataFrame({"num_clusters": num_clusters, "distortion": dist})
sns.lineplot(x="num_clusters", y="distortion", data=elbow_plot)


In [ ]:
cities_cluster = cities_cluster[['scaled_longitude', 'scaled_latitude']]
centroids, _ = kmeans(cities_cluster, 8)

cities_cluster['cluster_labels'], _ = vq(cities_cluster, centroids)

sns.scatterplot(x="scaled_longitude", y="scaled_latitude", data=cities_cluster, hue="cluster_labels")
plt.show()